# Importing all required liabraries

In [22]:
import pytesseract
import re
import os

# Data Extraction and Data Filtering

In [23]:
Invoice_no_list = []
Billing_date_list = []
Name_list = []
Mobile_no_list = []
location_list = []
Total_bill_list = []
Mode_of_Payment_list = []
final_of_final_list = []

folder_path =r"D:\my stuff\CDAC VITA\Python\project\invoice_details"
file_names_list = os.listdir(folder_path)

for image_name in file_names_list:
    image_name = os.path.join(folder_path,image_name)
    extracted_text = pytesseract.image_to_string(image_name)
    print(extracted_text)
    print("+"*50)
    extracted_text1 = list(extracted_text.split("\n"))
    final_list = []
    invoice_no = re.findall('\d{4}',extracted_text)[0]
    print(invoice_no)
    Invoice_no_list.append(invoice_no)
    final_list.append(invoice_no)
    
    x = extracted_text1[3]
    name_of = re.findall(r'[a-zA-Z]',x)
    name_of_patient = "".join(name_of)
    print(name_of_patient)
    Name_list.append(name_of_patient)
    final_list.append(name_of_patient)
    
    billing_date = re.findall('\d{2}/\d{2}/\d{4}',x)[0]
    print(billing_date)
    Billing_date_list.append(billing_date)
    final_list.append(billing_date)
    
    mobile_no = extracted_text1[4]
    print(mobile_no)
    Mobile_no_list.append(mobile_no)
    final_list.append(mobile_no)
    
    place = extracted_text1[5]
    print(place)
    location_list.append(place)
    final_list.append(place)
    
    total_amount = re.findall('\d{4}',extracted_text)[-2]
    print(total_amount)
    Total_bill_list.append(total_amount)
    final_list.append(total_amount)
    
    payment = re.findall('C...',extracted_text)
    for i in payment:
        if i=='Card' or i=='Cash':
            payment_mode = i
    final_list.append(payment_mode)
    Mode_of_Payment_list.append(payment_mode)
    print(payment_mode)
    print('*'*50)
    
    final_tuple = tuple(final_list)
    final_of_final_list.append(final_tuple)
    final_list.clear()

INVOICE

BILLED TO: Invoice No. 0001
Shubham G 19/05/2023
7774910157
Kolhapur
Item Unit Price Total
X-ray bill 2000 2000
Injections 1500 1500
Consultation fees 2000 2000
Subtotal 5500
Tax (0%) $0

Thank you!

PAYMENT INFORMATION
Payment Done by : Cash

Total 5500 Rs

Cooper Hospital
Juhu, Andheri(West) 4001053

++++++++++++++++++++++++++++++++++++++++++++++++++
0001
ShubhamG
19/05/2023
7774910157
Kolhapur
5500
Cash
**************************************************
INVOICE

BILLED TO: Invoice No. 0002
Anand Sant 19/05/2023
987654321
Amravati
Item Unit Price Total
Saline 2000 2000
Injections 1500 3000
Consultation fees 2000 2000
Subtotal 7000
Tax (0%) $0

Thank you!

PAYMENT INFORMATION
Payment Done by : Card

Total 7000 Rs

Cooper Hospital
Juhu, Andheri(West) 4001053

++++++++++++++++++++++++++++++++++++++++++++++++++
0002
AnandSant
19/05/2023
987654321
Amravati
7000
Card
**************************************************
INVOICE

BILLED TO: Invoice No. 0003
Dinesh Borole 18/05/2023
95

# Database Connectivity

### Database Table Creation

In [24]:
import mysql.connector as m
from tkinter import messagebox

mydatabase = m.connect(
    host='localhost',
    user='root',
    password='Rohit@150994',
    database='hospital_data'
)

cursor = mydatabase.cursor() 
drop_table_query = "DROP TABLE IF EXISTS invoice;"
cursor.execute(drop_table_query)
mydatabase.commit()

query = "create table invoice(invoice_no int primary key, patient_name varchar(30), invoice_date varchar(20), Mobile_No varchar(20), Location varchar(20), Total_Amount int, Mode_of_payment varchar(10));"
cursor.execute(query)
messagebox.showinfo("success","Table in database Hospital_Data created Successful", )

'ok'

### Data Insert in Database

In [25]:
for record in final_of_final_list:
    invoice_no,patient_name,invoice_date,Mobile_No,Location,Total_Amount,Mode_of_payment = record
    query = "insert into invoice(invoice_no,patient_name,invoice_date,Mobile_No,Location,Total_Amount,Mode_of_payment) values (%s,%s,%s,%s,%s,%s,%s)"
    values = (invoice_no,patient_name,invoice_date,Mobile_No,Location,Total_Amount,Mode_of_payment)
    cursor.execute(query,values)
    mydatabase.commit()
    
cursor.close()
mydatabase.close()
messagebox.showinfo("Success","Database succefully Updated")

'ok'

# Dataframe Creation

In [26]:
dict_of_data = {"Invoice No.":Invoice_no_list, "Billing date":Billing_date_list, "Name":Name_list, "Mobile No.":Mobile_no_list, "Location":location_list, "Total Bill":Total_bill_list, "Mode of Payment":Mode_of_Payment_list}

In [27]:
import pandas as pd

df = pd.DataFrame(dict_of_data)
df

,Invoice No.,Billing date,Name,Mobile No.,Location,Total Bill,Mode of Payment
0,0001,19/05/2023,ShubhamG,7774910157,Kolhapur,5500,Cash
1,0002,19/05/2023,AnandSant,987654321,Amravati,7000,Card
2,0003,18/05/2023,DineshBorole,9503363882,Jalgaon,8000,Card
3,0004,18/05/2023,SurajPatil,8600266356,Kalyan,7333,Card
4,0005,15/05/2023,SwarajSontakke,9112895842,Amravati,5555,Cash
5,0006,15/05/2023,AnshalTripathi,91319224094,Bilai,9565,Card
6,0007,15/05/2023,VivekHalpatrao,987321654,Kalyan,2565,Cash
7,0008,21/05/2023,PramodHatture,9890908040,Solapur,3000,Cash
8,0009,22/05/2023,AbhijeetDange,8806091540,Chandrapur,9888,Card


## Asking to create File

In [28]:
result = messagebox.askyesno("Confirm","Do you want to Export this data to CSV?")
if result:
    df.to_csv("Hospital_data.csv")
    messagebox.showinfo("Success","File Created successfully")
else:
    pass